# Algorithm

### a. Download the files from PBI Server
1. Download all files
2. Place them in the selected folder
3. Place the Jupyter notebook into the same folder
### b. Process the .pbix files - steps for each file
##### Preparation - create the temp directory for original archives contents
1. Unarchive the pbix file into the folder with the same name, which is located in the `temp` directory
2. Go to PBIX folder
3. Remove the `SecurityBindings` file
4. Remove SecurityBindings recors from the `[Content_Types].xml` (for instance, replace the file with the prepared one)
5. Modify the `Report/Layout` file - replace the values
6. Create the new archive using the PBIX folder contents
##### Remove the folder with Layouts
### c. Check the files and publish them to PBI server
1. Open each file
2. Save and publish

# Algorithm implementation

In [79]:
import os
import re
import calendar
import datetime
import json

from zipfile import ZipFile

#### Organizing the workspace. Creating the temporary and results folders

In [80]:
"""Path constants"""
WORKSPACE_PATH = os.getcwd()  # workspace directory with all the data
SOURCE_PATH = os.path.join(WORKSPACE_PATH, "source")  # directory with original .pbix files
TEMP_PATH = os.path.join(WORKSPACE_PATH, "temp")      # directory for temporary files
RESULTS_PATH = os.path.join(WORKSPACE_PATH, "result") # directory for the modified .pbix files

In [81]:
def create_workspace_hierarchy():
    """Checking if directories exist"""
    if not os.path.exists(RESULTS_PATH):
        os.mkdir(RESULTS_PATH)
    if not os.path.exists(TEMP_PATH):
        os.mkdir(TEMP_PATH)
    if not os.path.exists(SOURCE_PATH):
        os.mkdir(SOURCE_PATH)
        print("Please, place all the .pbix files into the 'source' directory, and then rerun the script")
    else:
        print("The directories structure is OK")

In [82]:
create_workspace_hierarchy()

The directories structure is OK


#### Iterating through files

1. Get the list of files in the `source` directory

In [83]:
source_pbix_files_names = [file_name[:-5] for file_name in os.listdir(SOURCE_PATH) if file_name[-5:] == ".pbix"]
print(source_pbix_files_names)

['Asia Regional Dashboard-GCL v3', 'test']


In [84]:
# Getting first element for algorithm example
pbix_file_name = source_pbix_files_names[0]
pbix_file_name

'Asia Regional Dashboard-GCL v3'

2. Unzip the .pbix file to the `temp` directory

In [85]:
# paths for the original .pbix file (archive) and temporary folder, to which the file will be unarchived
source_zip_path = os.path.join(SOURCE_PATH, pbix_file_name + ".pbix")
temp_dir_path = os.path.join(TEMP_PATH, pbix_file_name)

# opening the zip file in READ mode
with ZipFile(source_zip_path, 'r') as source_zip_file:
    print('Extracting all the files from', source_zip_path, "to", temp_dir_path)
    source_zip_file.extractall(path=temp_dir_path)

Extracting all the files from c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\source\Asia Regional Dashboard-GCL v3.pbix to c:\Users\Kateryna_Nemchenko\Desktop\Bookmark Updates\temp\Asia Regional Dashboard-GCL v3


3. Remove the `SecurityBindings` file

In [86]:
security_bindings_file_path = os.path.join(temp_dir_path, "SecurityBindings")
if os.path.exists(security_bindings_file_path):
    os.remove(security_bindings_file_path)

4. Remove SecurityBindings recors from the `[Content_Types].xml` (for instance, replace the file with the prepared one)

In [87]:
content_types_file_path = os.path.join(temp_dir_path, "[Content_Types].xml")

with open(content_types_file_path, "r") as content_types_file:
    xml = content_types_file.read()

updated_xml = xml.replace('<Override PartName="/SecurityBindings" ContentType="" />', "")

with open(content_types_file_path, "w") as content_types_file:
    content_types_file.write(updated_xml)

##### 5. Update the `Report/Layout` file

1. Month and year, to which the file should be updated

In [53]:
current_date = datetime.date.today()

current_year = current_date.year
current_month = current_date.month
current_month_abbr = calendar.month_abbr[current_month]

current_quarter = (current_month + 2) // 3

print(current_year, current_month, current_month_abbr, current_quarter)

2023 3 Mar 1


2. List of pattern and replacements pairs

In [71]:
PERIOD_REPLACEMENTS = [
    # pattern, new value
    ('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)', current_month_abbr),  # month eng
    ('\d{1,2}月', str(current_month) + "月"),   # month chn
    ('Q [1-4]', 'Q ' + str(current_quarter)),   # quarter eng (with space)
    ('Q[1-4]', 'Q' + str(current_quarter)),     # quarter eng (without space)
    #('\d{1,2}月', str(current_month) + "月"),  # quarter chn
    (str(current_year - 1), str(current_year))  # year
]

In [72]:
for (pattern, new_value) in PERIOD_REPLACEMENTS:
    print("new value - {}\t\tpattern - {}".format(new_value, pattern))

new value - Mar		pattern - (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)
new value - 3月		pattern - \d{1,2}月
new value - Q 1		pattern - Q [1-4]
new value - Q1		pattern - Q[1-4]
new value - 2023		pattern - 2022


3. Reading the `Layout` file

In [73]:
layout_file_path = os.path.join(temp_dir_path, "Report", "Layout")

with open(layout_file_path, "r", encoding="utf-16-le") as layout_file:
    layout_data = layout_file.read()

4. Functions for replacement

In [77]:
def replace_period(text:str, period_pattern:str, period_new_value:str):
    search_pattern = create_value_expression(period_pattern)
    new_value = create_value_expression(period_new_value)
    return re.sub(search_pattern, new_value, text)


def create_value_expression(value:str):
    return '"Value": "\'{}\'"'.format(value)

5. Replacing the patterns in the `Layout` file

In [78]:
for (pattern, new_value) in PERIOD_REPLACEMENTS:
    layout_data = replace_period(layout_data, pattern, new_value)

In [65]:
print(re.findall(r'"Value": "\'2023\'"', layout_data))

[]


##### 6. Assemble the new .pbix file

In [49]:
result_pbix_file_path = os.path.join(RESULTS_PATH, pbix_file_name + ".pbix")
                                     
with ZipFile(result_pbix_file_path, mode="w") as result_pbix_archive:
    for folderName, subfolders, filenames in os.walk(temp_dir_path):
        for filename in filenames:
            # create complete filepath of file in directory
            temp_dir_file_path = os.path.join(folderName, filename)
            # path for the archive hierarchy
            archive_path = temp_dir_file_path[len(temp_dir_path)+1:]
            # selecting the compression mode depending on archive file type
            file_compression_mode = result_pbix_archive.ZIP_STORED if filename == "DataModel" else result_pbix_archive.ZIP_DEFLATED
            # writing the file to result archive
            result_pbix_archive.write(temp_dir_file_path, arcname=archive_path, compress_type=file_compression_mode)

# The script